### import

In [1]:
# !gcloud auth login
# !gcloud auth application-default login
!gcloud config list

[artifacts]
location = asia-northeast3
repository = product-specialist
[core]
account = js.kim@hd-hyundai.com
disable_usage_reporting = True
project = hd-gen-ai-proc-391223

Your active configuration is: [default]


In [2]:
import os
from pathlib import Path
from typing import List, Dict, Any, Annotated, Optional
from dotenv import load_dotenv, find_dotenv

from typing_extensions import TypedDict
from langchain_google_vertexai import ChatVertexAI

# --- 1. 환경 설정 및 인증 정보 로드 ---
# root 경로의 .env 파일 load
# dotenv_path = Path(__file__).resolve().parent.parent.parent / '.env'
# load_dotenv(dotenv_path)  # .env 파일에서 환경 변수 로드
load_dotenv(find_dotenv())

# LangSmith 환경 변수 설정
os.environ["LANGSMITH_TRACING"] = os.getenv("LANGSMITH_TRACING", "false")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY", "")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT", "tsad-fed")

# Google Cloud 설정
PROJECT_ID = os.getenv("GOOGLE_CLOUD_PROJECT", "hd-gen-ai-proc-391223")
LOCATION = os.getenv("GOOGLE_CLOUD_REGION", "us-central1")
MODEL_NAME_G = os.getenv("GOOGLE_GEN_MODEL", "gemini-2.0-flash-001")

ModuleNotFoundError: No module named 'dotenv'

In [12]:
print(PROJECT_ID)

hd-gen-ai-proc-391223


### What to do.. ASIS prompt 부터 다시 살펴보기.

```python
response, raw_response, usage = get_gpt_response(args, prompt_res)
```

- `prompt_res`
```markdown
## Instructions
Determine if there are any anomalies in the provided AIOPS flow data sequence.

## Following Rules:
1. A data point is considered an anomaly if it is part of a sequence of at least one consecutive anomalous points or continues to plummet or surge abruptly. 
2. A data point is considered an anomaly if it is identified as a continuous low/high value anomaly if it remains below/above a predefined normal threshold for a prolonged duration, deviating from the anticipated norm.  
3. Given that the vast majority of data points are expected to be '''no anomaly''', Anomalies are exceedingly rare and should only be identified with absolute certainty.
4. Normal data may exhibit volatility, which should not be mistaken for anomalies. 
5. Mislabeling normal data as an anomaly can lead to catastrophic failures. Exercise extreme caution. False positives are unacceptable.
6. '''If do not have high percent confidence that data is an anomaly, do not flag it as an anomaly.'''
7. '''The output of anomaly intervals needs to be accurately located and should not be excessively long. '''
8. The number of abnormal intervals within a detection range can not exceed 3.
9. anomaly_type should be one of the following:
  - **PersistentLevelShiftUp**
    - The data shifts to a higher value and maintains that level consistently, '''do not return to the original baseline.''' like `1 2 1 2 1 2 *500* *480* *510* *500* *500*`
  - **PersistentLevelShiftDown**
    - The data shifts to a lower value and maintains that level consistently, '''do not return to the original baseline.''' like `1 2 1 2 *-100* *-102* *-104* *-110* *-110*`
  - **TransientLevelShiftUp**
    - The data temporarily shifts to a higher value and then returning to the original baseline, '''the anomaly maintains for at least 5 data points.''' like `1 2 1 2 1 2 *500* *500* *499* *510* *500* 1 2 1 2`
  - **TransientLevelShiftDown**
    - The data temporarily shifts to a lower value and then returning to the original baseline, '''the anomaly maintains for at least 5 data points.''' like `1 2 1 2 *-100* *-102* *-104* *-110* *-100* 1 2 1 2`
  - **SingleSpike**
    - A brief, sharp rise in data value followed by an immediate return to the baseline. like `1 2 1 2 1 2 *200* *500* 1 2`
  - **SingleDip**
    - A brief, sharp drop in data value followed by an immediate return to the baseline. like `1 2 1 2 *-500* *-200* 1 2 1 2`
  - **MultipleSpikes**
    - '''Several''' brief, sharp rises in data value, each followed by a return to the baseline. like `1 2 *500* 3 2 *510* *200* 1 2 *480* 1 2`
  - **MultipleDips**
    - '''Several''' brief, sharp drops in data value, each followed by a return to the baseline. like `1 2 *-100* 3 2 *-110* *-200* 1 2 *-120* 1 2`
10. alarm_level should be one of the following:
  - **Urgent/Error**
    - This category is for values that represent a severe risk, potentially causing immediate damage or harm across all event types whether increases, decreases, spikes, dips, or multiple occurrences.
  - **Important**
    - Allocated for moderate value changes (both increases and decreases) that could escalate to future problems or system stress but are not immediately hazardous. This also covers upward transient level shifts that concern system longevity and potential failure indications from downward shifts.
  - **Warning**
    - Used for noticeable deviations from the norm that are not yet critical but merit close monitoring. This includes single spikes and dips that are moderate in nature, as well as multiple non-critical spikes and level shifts that are significant but not yet dangerous.
11. The briefExplanation must comprise a explicit three-step analysis '''results''' utilizing precise data (do not only repeat the rule):
  - Step 1: Assess the overall trend to ascertain if it aligns with expected patterns, thereby identifying any overarching anomalies. 
  - Step 2: Determine if there is any local data segment with any continuous low or high values compared to the normal data sequence.
  - Step 3: Reassess the identified points to confirm their anomalous nature, given the rarity of true anomalies.
12. Provide responses in a strict JSON format suitable for direct parsing, without any additional textual commentary.

## Response Format
{
  "briefExplanation": {"step1_global": analysis reason, "step2_local": analysis reason, "step3_reassess": analysis reason},
  "is_anomaly": false/true,
  "anomalies": []/[index1, index2, index3, ...],
  "reason_for_anomaly_type": "no"/"reason for anomaly type",
  "anomaly_type": "no"/"classification of main anomaly",(only one)
  "reason_for_alarm_level": "no"/"reason for alarm level",
  "alarm_level": "no"/"Urgent/Error"/"Important"/"Warning",(only one)
}

## Data
Please analyze the latest data with the highest level of diligence and caution:
- Historical normal data sequence: `850,516,727,674,550,712,702,470,241,406,114,706,533,692,105,786,-1,1042,549,999,955,788,934,714,756,716,710,704,846,791,380,419,472,931,549,388,641,256,337,450,724,491,791,711,689,499,736,438,734,799,678,409,616,570,955,691,727,896,908,862,-55,406,914,882,748,970,568,747,793,691,738,522,622,593,725,993,625,898,699,1020,797,874,1042,299,752,881,525,741,710,696,546,883,827,568,683,655,669,553,711,575,262,520,178,559,4,731,924,505,729,952,622,836,629,733,704,750,618,582,448,658,575,571,570,720,329,819,698,135,811,768,544,449,987,679,548,558,584,528,715,604,518,511,656,536,561,528,899,942,484,1042,772,735,748,956,939,682,747,753,810,814,991,663,1007,726,537,519,560,666,611,174,765,297,1032,876,1037,752,720,833,830,573,580,616,876,820,800,693,779,645,779,393,392,1006,976,839,866,643,660,881,597,689,809,744,845,1007,540,499,821,742,583,746,563,713,907,1023,882,976,713,999,1025,666,927,687,588,822,787,621,800,760,962,767,1019,850,884,724,946,718,784,942,743,774,1027,834,712,714,805,713,594,1037,715,823,797,728,1000,700,872,610,18,1006,792,1038,1042,763,975,723,566,658,974,761,643,577,552,604,652,773,658,730,796,944,335,641,641,1033,879,778,748,709,1003,650,660,753,757,703,696,723,530,517,555,908,685,297,1042,784,975,984,1018,1019,964,1024,709,780,1032,695,513,540,740,559,792,954,615,741,802,841,808,942,549,869,815,846,770,724,775,674,845,1021,768,901,684,686,581,788,735,751,571,147,942,990,841,736,743,620,1002,864,827,763,687,826,667,941,672,596,657,726,662,794,942,953,942,835,847,1023,1005,689,648,688,614,683,657,708,918,423,663,648,869,510,947,991,779,889,1042,784,946,713,571,702,714,808,731,589,730,1022`

- Historical anomaly data sequence(*XXX* is anomaly point), `sequence 1: 924,882,916,804,891,894,925,910,884,778,*144*,512,811,764,636,734,668,415,*272*,711,*294*,472,878,769,860,808,849,888,836,894,915,881,800,559,473,684,773,741,685,764,802
sequence 2: 975,838,924,882,916,804,891,894,925,910,884,778,*144*,512,811,764,636,734,668,415,*272*,711,*294*,472,878,769,860,808,849,888,836,894,915,881,800,559,473,684,773,741,685
`
- The latest `400` data points for evaluation: `1 753
2 703
3 500
4 1028
5 554
6 1041
7 603

  ...

399 647
400 781
```

- 아래 부분이 핵심. 그 윗 부분은 모두 instruction 및 output format 지정
    - 그 중에서도 inference 데이터는 `cur_data` 혹은 prompt에서는 `data`
    - `anomaly_data`도 참고 용으로 사용 가능.
    - `normal_data`도 마찬가지.
    - 우선 3가지 데이터 모두 활용해도 되지만, 우선은 `cur_data`(400개 데이터 포인트, 벡터)를 먼저 활용하자.

```
## Data
Please analyze the latest data with the highest level of diligence and caution:
- Historical normal data sequence: `{normal_data}`
- Historical anomaly data sequence(*XXX* is anomaly point), `{anomaly_data}`
- The latest `{data_len}` data points for evaluation: `{data}`
```

### 시작

#### 판별 대상 데이터 샘플 분리
- 시작은 `data.value.tolist()`
    - `cur_data`의 원본 벡터 리스트
    - 이상치 탐지 대상 inference data


In [13]:
sample = [753, 703, 500, 1028, 554, 1041, 603, 676, 645, 599, 502, 463, 483, 475, 526, 496, 619, 418, 895, 498, 727, 1018, 756, 763, 600, 668, 816, 490, 721, 644, 642, 347, 638, 506, 605, 578, 528, 560, 626, 649, 485, 257, 486, 649, 919, 702, 874, 614, 614, 469, 699, 430, 553, 469, 496, 934, 518, 597, 696, 602, 564, 509, 670, 775, 611, 874, 794, 613, 478, 657, 679, 644, 557, 567, 490, 685, 662, 511, 618, 606, 692, 308, 657, 583, 675, 736, 766, 811, 1042, 842, 547, 402, 1032, 598, 690, 643, 515, 621, 490, 550, 530, 500, 602, 679, 577, 573, 592, 644, 869, 811, 811, 766, 1042, 728, 527, 636, 663, 710, 297, 564, 772, 720, 687, 637, 491, 1041, 543, 518, 998, 342, 196, 702, 976, 702, 914, 891, 658, 636, 708, 1028, 743, 837, 517, 730, 607, 529, 568, 461, 598, 654, 726, 887, 356, 1042, 702, 530, 735, 691, 539, 657, 595, 509, 660, 628, 588, 631, 359, 442, 677, 619, 774, 668, 598, 623, 595, 825, 356, 725, 841, 517, 566, 516, 524, 925, 545, 665, 537, 425, 505, 559, 484, 520, 572, 663, 758, 920, 884, 818, 748, 171, 595, 464, 441, 622, 733, 543, 591, 582, 364, 562, 522, 566, 674, 633, 374, 542, 942, 876, 1006, 844, 716, 468, 555, 589, 698, 419, 525, 614, 436, 613, 691, 650, 594, 603, 596, 240, 839, 942, 702, 1023, 935, 938, 567, 790, 607, 758, 617, 577, 619, 620, 951, 752, 660, 493, 664, 545, 643, 613, 427, 999, 1024, 869, 614, 976, 869, 711, 891, 664, 783, 756, 793, 621, 833, 810, 729, 607, 655, 662, 930, 747, 674, 600, 544, 775, 695, 711, 542, 702, 944, 845, 652, 915, 710, 703, 884, 769, 701, 746, 765, 771, 751, 659, 674, 730, 702, 732, 1042, 869, 862, 1042, 942, 614, 570, 639, 685, 614, 599, 428, 635, 762, 632, 575, 810, 654, 659, 758, 538, 640, 600, 580, 914, 881, 811, 1031, 807, 614, 886, 626, 642, 668, 742, 739, 721, 502, 606, 644, 812, 582, 671, 715, 640, 653, 942, 784, 784, 631, 702, 817, 654, 760, 617, 514, 683, 667, 542, 730, 573, 681, 594, 609, 502, 599, 865, 931, 838, 675, 804, 627, 646, 757, 689, 736, 996, 761, 710, 595, 560, 657, 664, 705, 646, 671, 668, 666, 702, 708, 645, 786, 647, 781]
sample_str = [str(i) for i in sample]
print(sample_str)

['753', '703', '500', '1028', '554', '1041', '603', '676', '645', '599', '502', '463', '483', '475', '526', '496', '619', '418', '895', '498', '727', '1018', '756', '763', '600', '668', '816', '490', '721', '644', '642', '347', '638', '506', '605', '578', '528', '560', '626', '649', '485', '257', '486', '649', '919', '702', '874', '614', '614', '469', '699', '430', '553', '469', '496', '934', '518', '597', '696', '602', '564', '509', '670', '775', '611', '874', '794', '613', '478', '657', '679', '644', '557', '567', '490', '685', '662', '511', '618', '606', '692', '308', '657', '583', '675', '736', '766', '811', '1042', '842', '547', '402', '1032', '598', '690', '643', '515', '621', '490', '550', '530', '500', '602', '679', '577', '573', '592', '644', '869', '811', '811', '766', '1042', '728', '527', '636', '663', '710', '297', '564', '772', '720', '687', '637', '491', '1041', '543', '518', '998', '342', '196', '702', '976', '702', '914', '891', '658', '636', '708', '1028', '743', '837

### 방향 - federation, mixture of agents, supervised multi-agents

- ref: https://github.com/langchain-ai/langgraph/blob/main/docs/docs/tutorials/multi_agent/agent_supervisor.ipynb

#### 1. tool 정의

1. ts2img
2. math_tools(llm_compiler 활용)
3. etc..

In [14]:
from langchain_core.tools import tool

@tool
def ts2img(data: List[float], title: str = "Time Series Plot") -> str:
    """
    Generates a line plot from a list of numbers and saves it as a local PNG image.

    Args:
        data (List[float]): A list of numerical time series data to plot.
        title (str, optional): The title for the plot. Defaults to "Time Series Plot".

    Returns:
        str: A JSON string with status, message, and image path information.
    """
    try:
        if not data:
            raise ValueError("Input data list is empty")
        
        data_np = np.array(data)
        fig, ax = plt.subplots(figsize=(10, 4))
        ax.plot(data_np, linewidth=1.5)
        ax.set_title(title, fontsize=14)
        ax.set_xlabel("Time Index", fontsize=10)
        ax.set_ylabel("Value", fontsize=10)
        ax.grid(True, linestyle='--', alpha=0.6)
        
        # Create directory for images if it doesn't exist
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
        img_dir = Path(__file__).resolve().parent.parent / "temp_images"
        img_dir.mkdir(parents=True, exist_ok=True)
        save_path = img_dir / f"timeseries_{timestamp}.png"
        
        plt.savefig(save_path, dpi=100, bbox_inches='tight')
        plt.close(fig)
        
        print(f"Image saved to {save_path}")
        return json.dumps({
            "status": "success", 
            "message": f"Image generated: {save_path.name}", 
            "image_path": str(save_path)
        })
    
    except Exception as e:
        error_message = f"Error generating image: {e}"
        print(error_message)
        return json.dumps({"status": "error", "message": error_message})

In [ ]:
from study.math_tools import get_math_tool

calculate = get_math_tool(ChatVertexAI(model=MODEL_NAME_G))

In [ ]:
calculate.invoke(
    {
        "problem": "What's the temp of sf + 5?",
        "context": ["Thet empreature of sf is 32 degrees"],
    }
)